# Imports

In [1]:
%pip install --upgrade pip
%pip install transformers
%pip install accelerate
%pip install datasets
%pip install bitsandbytes
%pip install einops
# For the peft library, since it's from a git repository:
%pip install git+https://github.com/huggingface/peft.git
%pip install peft
%pip install trl
%pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/peft.git to /tmp/

In [3]:
from datasets import load_dataset
from random import randrange

#import torch
from transformers import AutoTokenizer,AutoModelForCausalLM,AutoModelForSeq2SeqLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM
from trl import SFTTrainer #supervised finetuning
from huggingface_hub import login

ImportError: libnccl.so.2: cannot open shared object file: No such file or directory

In [ ]:
model_name = "EleutherAI/polyglot-ko-1.3b"

# Dataset

In [ ]:
def prompt_instruction_format(sample):
  return f""" ### Instruction:
    Use the task below and the input given to write the response:

    ### Task:
    Give response like a teacher to the input dialouge of student and teacher

    ### Input:
    {sample['Dialogue']}

    ### Reponse:
    {sample['Response']}
    """

In [ ]:
from datasets import Dataset
import pandas as pd
test = pd.read_excel('test.xlsx')
dataset_test = Dataset.from_pandas(test)

train = pd.read_excel('train.xlsx')
dataset= Dataset.from_pandas(train)

# dataset = {'train': dataset_train, 'test': dataset_test}

In [ ]:
from transformers import AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXConfig

#Loading the Pre-trained Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPTNeoXForCausalLM.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token #end of sentence tokenizer
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
model-00002-of-00003.safetensors:   3%|▎         | 31.5M/1.02G [00:04<02:28, 6.63MB/s]


KeyboardInterrupt: 

In [ ]:
from transformers import  TrainingArguments

# trainingArgs = TrainingArguments(
#     output_dir = '/output',
#     num_train_epochs = 1,
#     per_device_train_batch_size = 4,
#     save_strategy= "epoch",
#     learning_rate = 2e-4,
#     optim="paged_adamw_32bit",
#     logging_steps=1,
# )
trainingArgs = TrainingArguments(
    output_dir = '/output',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit", # from the QLoRA paper
    logging_steps=1,
    save_strategy="epoch",
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#create the trainer
peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM" #predicting the next token in the sequence
)

max_seq_length = 2048

trainer = SFTTrainer(
    model= model,
    train_dataset=dataset.select(range(300)),
    peft_config = peft_config,
    #max_seq_length=max_seq_length,
    tokenizer = tokenizer,
    packing=True,
    formatting_func = prompt_instruction_format,
    args= trainingArgs,
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:222: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
trainer.train()


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 2.3431, 'learning_rate': 0.0002, 'epoch': 0.12}


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 2.2922, 'learning_rate': 0.0002, 'epoch': 0.24}


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


...

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 1.7292, 'learning_rate': 0.0002, 'epoch': 0.82}


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


{'loss': 1.7553, 'learning_rate': 0.0002, 'epoch': 0.94}
{'train_runtime': 163.7869, 'train_samples_per_second': 0.403, 'train_steps_per_second': 0.049, 'train_loss': 1.9740640819072723, 'epoch': 0.94}


TrainOutput(global_step=8, training_loss=1.9740640819072723, metrics={'train_runtime': 163.7869, 'train_samples_per_second': 0.403, 'train_steps_per_second': 0.049, 'train_loss': 1.9740640819072723, 'epoch': 0.94})

In [ ]:
trainer.save_model()

In [ ]:
from peft import AutoPeftModelForCausalLM

# Load the trained model
model = AutoPeftModelForCausalLM.from_pretrained(
    trainingArgs.output_dir,  # Ensure this is the correct path
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def generate_response(sample_index):
    # Get a sample from the dataset
    sample = dataset[sample_index]

    # Format the prompt
    prompt = prompt_instruction_format(sample)

    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

    # Make sure the model and inputs are on the GPU
    model = model.to('cuda')
    input_ids = input_ids.to('cuda')

    # Generate the model's response
    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=500,
        min_length=50,
        do_sample=True,
        top_k=50,
        top_p=0.85,
        temperature=0.7,
        num_return_sequences=1
    )

    # Decode the generated response
    generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the results
    print(f"Sample {sample_index}:")
    print(f"Prompt:\n{prompt}\n")
    print(f"-------------")
    print(f"Generated Response:\n{generated_response}")
    print(f"-------------")
    print(f"Ground truth:\n{sample['Response']}\n")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


Prompt:
 ### Instruction:
    Use the taks below and the input given to write the response:

    ### Task:
    Give response to the input dialouge

    ### Input:
    Student: 우선 내신이나 정시같이 공부를 우선으로 해서

    ### Reponse:
    Teacher: 네~!
    

-------------
Generated Response:
 ### Instruction:
    Use the taks below and the input given to write the response:

    ### Task:
    Give response to the input dialouge

    ### Input:
    Student: 우선 내신이나 정시같이 공부를 우선으로 해서

    ### Reponse:
    Teacher: 네~!
     ### Input::     Teacher: 네~! ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
-------------
Ground truth:
Teacher: 네~!


In [ ]:
generate_response(5)


In [ ]:
generate_response(10)


In [ ]:
generate_response(14)


In [ ]:
generate_response(20)


# response from untuned model

In [ ]:
def generate_untuned_response(sample_index):
    # Load the untuned model and tokenizer
    untuned_model = AutoModelForCausalLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    tokenizer.pad_token = tokenizer.eos_token #end of sentence tokenizer
    tokenizer.padding_side = "right"

    # Get a sample from the dataset
    sample = dataset[sample_index]

    # Format the prompt
    prompt = prompt_instruction_format(sample)

    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids


    # If you're using a GPU, you can uncomment the following line
    input_ids = input_ids.cuda()

    # Make sure the model and inputs are on the GPU
    untuned_model = untuned_model.to('cuda')

    # Generate the model's response
    outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=500,
        min_length=50,
        do_sample=True,
        top_k=50,
        top_p=0.85,
        temperature=0.7,
        num_return_sequences=1
    )

    # Decode the generated response
    generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the results
    print(f"Sample {sample_index}:")
    print(f"Prompt:\n{prompt}\n")
    print(f"-------------")
    print(f"Generated Response:\n{generated_output}")
    print(f"-------------")
    print(f"Ground truth:\n{sample['Response']}\n")



In [ ]:
generate_untuned_response(5)

NameError: name 'AutoModelForCausalLM' is not defined

In [ ]:
generate_untuned_response(10)

In [ ]:
generate_untuned_response(14)

In [ ]:
generate_untuned_response(20)